<h2>Find coordinates of neighborhoods', boroughs' centers and add 'hover' data</h2>

In [37]:
from geojson_utils import point_in_multipolygon
from geojson_utils import centroid
import pandas as pd
import json

In [38]:
dataN= pd.read_json('data/NeighborhoodTabulationAreas.geojson')
dataB = pd.read_json('data/BoroughBoundaries.geojson')

In [39]:
centersN = pd.DataFrame(columns=['ntaname', 'centerLong', 'centerLat'])
centersB = pd.DataFrame(columns=['boroname', 'centerLong', 'centerLat'])
for element in dataN['features']:
    box_str = str(element['geometry']['coordinates'][0])
    box_str = '{"type": "Polygon","coordinates": '+box_str+'}'
    box = json.loads(box_str)
    point = centroid(box)
    centersN = centersN.append({'ntaname': element['properties']['ntaname'].lower(), 'centerLong': point['coordinates'][0], 'centerLat': point['coordinates'][1]}, ignore_index=True)
    
for element in dataB['features']:
    box_str = str(element['geometry']['coordinates'][0])
    box_str = '{"type": "Polygon","coordinates": '+box_str+'}'
    box = json.loads(box_str)
    point = centroid(box)
    centersB = centersB.append({'boroname': element['properties']['boro_name'].lower(), 'centerLong': point['coordinates'][0], 'centerLat': point['coordinates'][1]}, ignore_index=True)    

In [40]:
centersN

,ntaname,centerLong,centerLat
0,borough park,-73.988661,40.630950
1,murray hill,-73.809546,40.768352
2,east elmhurst,-73.868396,40.763352
3,hollis,-73.761137,40.710639
4,manhattanville,-73.953782,40.817976
...,...,...,...
190,ozone park,-73.847049,40.675599
191,lindenwood-howard beach,-73.858961,40.652852
192,university heights-morris heights,-73.915974,40.852522
193,new brighton-silver lake,-74.102861,40.630050


In [41]:
centersB

,boroname,centerLong,centerLat
0,bronx,-73.897843,40.796350
1,staten island,-74.050069,40.565911
2,brooklyn,-73.866373,40.586820
3,queens,-73.837516,40.592832
4,manhattan,-74.010173,40.685134


In [42]:
dfN = pd.read_csv('data/MergedTreesPropertiesData.csv')
dfB = pd.read_csv('data/MergedTreesPropertiesDataBoro.csv')
centersB.rename(columns={'boroname': 'borough'}, inplace=True)
dfB.head()

,borough,health,trees/sq.mile,avg.landprice_thous$/acre,properties/sq.mile
0,bronx,13.897568,2575.768919,56.952162,3636.815946
1,brooklyn,13.806863,2840.467843,37.254118,5551.347255
2,manhattan,13.460000,3307.277692,219.828846,8028.133846
3,queens,13.878929,2678.568036,35.663571,4278.410714
4,staten island,13.924444,2130.842778,24.254444,2752.206111


In [43]:
dfN = pd.merge(dfN, centersN, how='outer', on='ntaname')
dfB = pd.merge(dfB, centersB, how='outer', on='borough')

In [45]:
dfN.dropna(inplace=True)
dfB.dropna(inplace=True)


In [46]:
dfN['hover']=''
dfB['hover']=''

In [47]:
for index, row in dfN.iterrows():
    new_hover=row['ntaname']+'<br>'+row['borough']+'<br>'+'trees/sq.mile: '+str(row['trees/sq.mile'])+'<br>'+'avg. landprice: '+ str(row['avg.landprice_thous$/acre']) + '<br>'+'trees health: '+str(row['health'])+'/15'
    dfN.at[index,'hover'] = new_hover

for index, row in dfB.iterrows():
    new_hover=row['borough']+'<br>'+'trees/sq.mile: '+str(row['trees/sq.mile'])+'<br>'+'avg. landprice: '+ str(row['avg.landprice_thous$/acre']) + '<br>'+'trees health: '+str(row['health'])+'/15'
    dfB.at[index,'hover'] = new_hover

In [48]:
dfN.drop(columns={'borough_y', 'avtot'}, inplace = True)

In [49]:
dfN.hover[0]

'airport<br>queens<br>trees/sq.mile: 18.94<br>avg. landprice: 1995.03<br>trees health: 14.32/15'

In [50]:
dfN.to_csv('data\Trees_Properties_With_Centroids.csv')

In [51]:
dfB.to_csv('data\Trees_Properties_With_Centroids_Boro.csv')